In [0]:
df = spark.read.format("delta").load('abfss://bronze@databricksetestg.dfs.core.windows.net/products')
df = df.drop('_rescued_data')
display(df)

In [0]:
df.createOrReplaceTempView('products')

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%sql
CREATE OR REPLACE FUNCTION prem_catalog.bronze.discount_func(p_price DOUBLE)
RETURNS double
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql
CREATE OR REPLACE FUNCTION prem_catalog.bronze.brand_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
 return p_brand.upper()
$$

In [0]:
df = df.withColumn('discount_price',expr('prem_catalog.bronze.discount_func(price)'))
df = df.withColumn('brand_upper',expr('prem_catalog.bronze.brand_func(brand)'))


In [0]:
df.show()

In [0]:
%sql
select product_id, price , prem_catalog.bronze.discount_func(price) as discounted_price from products

In [0]:
df.write.format('delta').mode('overwrite').save('abfss://silver@databricksetestg.dfs.core.windows.net/products')

In [0]:
%sql
CREATE TABLE IF NOT EXISTS prem_catalog.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@databricksetestg.dfs.core.windows.net/products'